## Parte IV: Consignas adicionales
### Visualizacion de un embedding(word2vec)
Bautista Boeri - 110898

Enlace a este notebook:https://colab.research.google.com/drive/10JqNrtAdUUQJwnCjpuRGiUQnSo5lZ_uI?usp=sharing

## 1. Imports y Configuración

In [ ]:
!pip install -q gensim plotly umap-learn nltk nbformat>=4.2.0

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
import umap
from gensim.models import Word2Vec
import re
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))

print("✓ Librerías importadas correctamente")

✓ Librerías importadas correctamente


## 2. Cargar y Preparar Datos

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

# Cargar dataset
tweets_df = pd.read_pickle('/content/drive/MyDrive/tp3/tweets_engineered_features_refactored.pkl')

print(f"Dataset shape: {tweets_df.shape}")
print(f"\nDistribución de clases:")
print(tweets_df['target'].value_counts())
display(tweets_df.head())

Mounted at /content/drive
Dataset shape: (7613, 12)

Distribución de clases:
target
0    4342
1    3271
Name: count, dtype: int64


,id,keyword,location,text,target,cantidad_emoji,cantidad_hashtags,cantidad_arrobas,sentimiento,cantidad_links,tiene_4_simbolos_consecutivos,cantidad_palabras
0,1,sin_keyword,sin_locacion,Our Deeds are the Reason of this #earthquake M...,1,0,1,0,positivo,0,0,13
1,4,sin_keyword,sin_locacion,Forest fire near La Ronge Sask. Canada,1,0,0,0,negativo,0,0,7
2,5,sin_keyword,sin_locacion,All residents asked to 'shelter in place' are ...,1,0,0,0,negativo,0,0,22
3,6,sin_keyword,sin_locacion,"13,000 people receive #wildfires evacuation or...",1,0,1,0,positivo,0,0,8
4,7,sin_keyword,sin_locacion,Just got sent this photo from Ruby #Alaska as ...,1,0,2,0,positivo,0,0,16


## 3. Limpieza de Texto

In [ ]:
def limpiar_texto(texto):
    """Limpia el texto para Word2Vec"""
    texto = str(texto).lower()
    texto = re.sub(r'http\S+|www.\S+', '', texto)
    texto = re.sub(r'@\w+', '', texto)
    texto = re.sub(r'#', '', texto)
    texto = re.sub(r'[^a-z\s]', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

# Limpiar textos
tweets_df['text_clean'] = tweets_df['text'].apply(limpiar_texto)



## 4. Entrenar Word2Vec

Word2Vec aprende representaciones vectoriales de palabras basándose en el contexto en que aparecen. Usaremos el algoritmo **Skip-gram** que predice palabras del contexto dado una palabra central.

In [ ]:
# Preparar corpus: lista de listas de palabras, excluyendo stopwords
corpus = [[word for word in text.split() if word not in stop_words] for text in tweets_df['text_clean']]

print(f"Total de tweets: {len(corpus)}")
print(f"Ejemplo de tweet tokenizado: {corpus[0]}")

# Entrenar Word2Vec
print("\n🚀 Entrenando Word2Vec...")
w2v_model = Word2Vec(
    sentences=corpus,
    vector_size=100,      # Dimensiones del vector
    window=5,             # Contexto: 5 palabras antes y después
    min_count=5,          # Ignorar palabras con frecuencia < 5
    workers=4,            # Threads paralelos
    seed=42,
    epochs=10
)

print(f"✓ Modelo entrenado con {len(w2v_model.wv)} palabras en el vocabulario")

Total de tweets: 7613
Ejemplo de tweet tokenizado: ['deeds', 'reason', 'earthquake', 'may', 'allah', 'forgive', 'us']

🚀 Entrenando Word2Vec...
✓ Modelo entrenado con 2635 palabras en el vocabulario


## 5. Seleccionar Palabras para Visualización

Seleccionaremos las palabras más frecuentes y relevantes para visualizar sus embeddings.

In [ ]:
# Obtener palabras más frecuentes
from collections import Counter

todas_palabras = [palabra for tweet in corpus for palabra in tweet]
frecuencias = Counter(todas_palabras)

print(f"Total de palabras (con repeticiones): {len(todas_palabras)}")
print(f"Vocabulario único: {len(frecuencias)}")
print(f"\nPalabras más frecuentes:")
for palabra, freq in frecuencias.most_common(20):
    print(f"  {palabra}: {freq}")

Total de palabras (con repeticiones): 66542
Vocabulario único: 13882

Palabras más frecuentes:
  like: 348
  amp: 344
  fire: 254
  get: 229
  new: 227
  via: 220
  news: 207
  people: 199
  one: 199
  video: 165
  disaster: 158
  emergency: 158
  police: 143
  would: 137
  u: 137
  body: 131
  time: 130
  still: 129
  us: 126
  california: 121


## 6. Seleccionar Palabras y Vectores para Visualización

Seleccionaremos las palabras más frecuentes para visualizar:

In [ ]:
# Seleccionar top 500 palabras más frecuentes para visualizar
top_n = 500
palabras_visualizar = [palabra for palabra, freq in frecuencias.most_common(top_n)]

# Obtener vectores de esas palabras
vectores = np.array([w2v_model.wv[palabra] for palabra in palabras_visualizar])

print(f"✓ Seleccionadas {len(palabras_visualizar)} palabras")
print(f"✓ Shape de vectores: {vectores.shape}")

# Palabras relacionadas con desastres para etiquetar
palabras_desastre = [
    'fire', 'earthquake', 'flood', 'disaster', 'emergency', 'bomb', 'crash',
    'killed', 'death', 'destroy', 'attack', 'storm', 'victims', 'damage',
    'rescue', 'police', 'accident', 'burning', 'explosion', 'threat',
    'evacuation', 'injuries', 'collapsed', 'panic', 'terror', 'warning'
]

print(f"\nPalabras de desastre en vocabulario: {sum(1 for p in palabras_desastre if p in palabras_visualizar)}")

✓ Seleccionadas 500 palabras
✓ Shape de vectores: (500, 100)

Palabras de desastre en vocabulario: 23


In [ ]:
# Calcular tendencia de cada palabra según target
# Para cada palabra, contar cuántas veces aparece en tweets de desastre vs no desastre

palabra_tendencia = {}

for palabra in palabras_visualizar:
    count_desastre = 0  # target = 1
    count_no_desastre = 0  # target = 0

    for idx, tweet in enumerate(corpus):
        if palabra in tweet:
            if tweets_df.iloc[idx]['target'] == 1:
                count_desastre += 1
            else:
                count_no_desastre += 1

    # Calcular ratio: valores cercanos a 1 = más en desastres, cercanos a 0 = más en no-desastres
    total = count_desastre + count_no_desastre
    if total > 0:
        palabra_tendencia[palabra] = count_desastre / total
    else:
        palabra_tendencia[palabra] = 0.5  # neutral si no aparece

print(f"✓ Tendencia calculada para {len(palabra_tendencia)} palabras")
print(f"\nEjemplos de palabras con alta tendencia a desastre:")
top_desastre = sorted(palabra_tendencia.items(), key=lambda x: x[1], reverse=True)[:10]
for palabra, tendencia in top_desastre:
    print(f"  {palabra}: {tendencia:.2%}")

print(f"\nEjemplos de palabras con baja tendencia a desastre:")
top_no_desastre = sorted(palabra_tendencia.items(), key=lambda x: x[1])[:10]
for palabra, tendencia in top_no_desastre:
    print(f"  {palabra}: {tendencia:.2%}")

✓ Tendencia calculada para 500 palabras

Ejemplos de palabras con alta tendencia a desastre:
  northern: 100.00%
  legionnaires: 100.00%
  debris: 100.00%
  severe: 100.00%
  derailment: 100.00%
  migrants: 100.00%
  investigators: 100.00%
  mosque: 100.00%
  pkk: 100.00%
  detonated: 100.00%

Ejemplos de palabras con baja tendencia a desastre:
  bags: 2.50%
  bag: 2.56%
  ruin: 2.63%
  ebay: 3.33%
  blew: 5.88%
  panicking: 6.06%
  blazing: 6.45%
  song: 6.67%
  shoulder: 6.90%
  wrecked: 7.69%


## 7. Reducción de Dimensionalidad con t-SNE

**t-SNE** (t-Distributed Stochastic Neighbor Embedding) reduce 100 dimensiones a 2D preservando las distancias locales entre puntos.

In [ ]:
print("Aplicando t-SNE (esto puede tardar un minuto)...")

tsne = TSNE(
    n_components=2,
    perplexity=30,
    max_iter=1000,
    random_state=42
)

vectores_2d = tsne.fit_transform(vectores)

print(f"✓ Reducción completada: {vectores_2d.shape}")

Aplicando t-SNE (esto puede tardar un minuto)...
✓ Reducción completada: (500, 2)


## 7.1 Preparar DataFrame para Visualización

In [ ]:
# Crear DataFrame para visualización
df_viz = pd.DataFrame({
    'palabra': palabras_visualizar,
    'x': vectores_2d[:, 0],
    'y': vectores_2d[:, 1],
    'frecuencia': [frecuencias[p] for p in palabras_visualizar],
    'tendencia_desastre': [palabra_tendencia[p] for p in palabras_visualizar]
})

# Crear categoría para mejor visualización
df_viz['categoria'] = df_viz['tendencia_desastre'].apply(
    lambda x: 'Desastre (>70%)' if x > 0.7 else ('No desastre (<30%)' if x < 0.3 else 'Neutral')
)

print(f"✓ DataFrame creado con {len(df_viz)} palabras")
print(f"\nDistribución por categoría:")
print(df_viz['categoria'].value_counts())

✓ DataFrame creado con 500 palabras

Distribución por categoría:
categoria
Neutral               222
No desastre (<30%)    156
Desastre (>70%)       122
Name: count, dtype: int64


## 8. Visualización Interactiva con t-SNE

In [ ]:
# Crear gráfico interactivo coloreado por target
fig = px.scatter(
    df_viz,
    x='x',
    y='y',
    color='tendencia_desastre',
    hover_data=['palabra', 'frecuencia', 'tendencia_desastre'],
    title='Embeddings Word2Vec - t-SNE (coloreado por tendencia a desastre)',
    labels={
        'x': 't-SNE Dimensión 1',
        'y': 't-SNE Dimensión 2',
        'tendencia_desastre': 'Tendencia Desastre'
    },
    color_continuous_scale='RdYlBu_r',  # Rojo=Desastre, Azul=No desastre
    width=1000,
    height=700
)

# Agregar etiquetas para palabras clave
palabras_etiquetar = [p for p in palabras_desastre if p in palabras_visualizar][:15]
df_etiquetas = df_viz[df_viz['palabra'].isin(palabras_etiquetar)]

for _, row in df_etiquetas.iterrows():
    fig.add_annotation(
        x=row['x'],
        y=row['y'],
        text=row['palabra'],
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=1,
        arrowcolor='gray',
        ax=20,
        ay=-30,
        font=dict(size=9, color='black'),
        bgcolor='rgba(255,255,255,0.8)'
    )

fig.update_traces(marker=dict(size=8, opacity=0.7))
fig.show()

## 9. Reducción con UMAP

**UMAP** (Uniform Manifold Approximation and Projection) es una alternativa a t-SNE que a veces preserva mejor la estructura global.

In [ ]:
print("Aplicando UMAP...")

reducer = umap.UMAP(
    n_components=2,
    n_neighbors=15,
    min_dist=0.1,
    metric='cosine',
    random_state=42
)

vectores_umap = reducer.fit_transform(vectores)

# Actualizar DataFrame
df_viz['x_umap'] = vectores_umap[:, 0]
df_viz['y_umap'] = vectores_umap[:, 1]

print(f"✓ UMAP completado: {vectores_umap.shape}")

Aplicando UMAP...
✓ UMAP completado: (500, 2)


In [ ]:
# Visualización UMAP coloreada por target
fig2 = px.scatter(
    df_viz,
    x='x_umap',
    y='y_umap',
    color='tendencia_desastre',
    hover_data=['palabra', 'frecuencia', 'tendencia_desastre'],
    title='Embeddings Word2Vec - UMAP (coloreado por tendencia a desastre)',
    labels={
        'x_umap': 'UMAP Dimensión 1',
        'y_umap': 'UMAP Dimensión 2',
        'tendencia_desastre': 'Tendencia Desastre'
    },
    color_continuous_scale='RdYlBu_r',  # Rojo=Desastre, Azul=No desastre
    width=1000,
    height=700
)

# Agregar etiquetas
df_etiquetas_umap = df_viz[df_viz['palabra'].isin(palabras_etiquetar)]

for _, row in df_etiquetas_umap.iterrows():
    fig2.add_annotation(
        x=row['x_umap'],
        y=row['y_umap'],
        text=row['palabra'],
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=1,
        arrowcolor='gray',
        ax=20,
        ay=-30,
        font=dict(size=9, color='black'),
        bgcolor='rgba(255,255,255,0.8)'
    )

fig2.update_traces(marker=dict(size=8, opacity=0.7))
fig2.show()

## 10. Visualización 3D con UMAP

Visualicemos en 3 dimensiones para capturar más estructura.

In [ ]:
print("Aplicando UMAP 3D...")

reducer_3d = umap.UMAP(
    n_components=3,
    n_neighbors=15,
    min_dist=0.1,
    metric='cosine',
    random_state=42
)

vectores_3d = reducer_3d.fit_transform(vectores)

df_viz['x_3d'] = vectores_3d[:, 0]
df_viz['y_3d'] = vectores_3d[:, 1]
df_viz['z_3d'] = vectores_3d[:, 2]

print("✓ UMAP 3D completado")

Aplicando UMAP 3D...
✓ UMAP 3D completado


In [ ]:
# Gráfico 3D interactivo coloreado por target
fig4 = px.scatter_3d(
    df_viz,
    x='x_3d',
    y='y_3d',
    z='z_3d',
    color='tendencia_desastre',
    hover_data=['palabra', 'frecuencia', 'tendencia_desastre'],
    title='Embeddings Word2Vec - 3D UMAP (coloreado por tendencia a desastre)',
    labels={
        'x_3d': 'UMAP Dim 1',
        'y_3d': 'UMAP Dim 2',
        'z_3d': 'UMAP Dim 3',
        'tendencia_desastre': 'Tendencia Desastre'
    },
    color_continuous_scale='RdYlBu_r',  # Rojo=Desastre, Azul=No desastre
    width=1000,
    height=800
)

fig4.update_traces(marker=dict(size=4, opacity=0.8))
fig4.show()
print("\n💡 Rota el gráfico para explorar diferentes ángulos")


💡 Rota el gráfico para explorar diferentes ángulos


## Podemos predecir el target con los graficos anteriores?

Es dificil asegurar el target a partir de los graficos anteriores pues algunas palabras son utilizadas por tweets que son desatres reales como otros que no. Igualmente, podemos observar, especialmente en el 3d como hay concentraciones de puntos rojos. Asi como hay de unos azules. EL problema es que hay palabras mas intermedias que no tienen un significado muy relevante pero que son utilizadas multiples veces. Ademas, notemos que un tweet posee multiples palabras por lo que tambien podriamos tener una combinacion de palabras que podria estar segmentadas a lo largo del grafico con distintas tonalidades de colores. Por ejemplo: si tenemos un tweets con varias palabras que estan en algun cluster con una concentracion de puntos rojos, es probable que sea un desastre real. Igualmente, esto no es garantia para predecir el target pero si podria darnos una idea general